<a href="https://colab.research.google.com/github/Gopichand14052004/BATCH_5_NLP/blob/main/2203a51015_ass_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Step (a): Define small dataset
data = [("hello", "bonjour"), ("how are you", "comment ça va"), ("I am fine", "je vais bien"),
        ("what is your name", "comment tu t'appelles"), ("my name is", "je m'appelle"),
        ("thank you", "merci"), ("goodbye", "au revoir")]

# Preprocess data
input_texts = [pair[0] for pair in data]
target_texts = ['\t' + pair[1] + '\n' for pair in data]  # \t is start token, \n is end token

input_chars = sorted(set("".join(input_texts)))
target_chars = sorted(set("".join(target_texts)))
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

input_token_index = dict([(char, i) for i, char in enumerate(input_chars)])
target_token_index = dict([(char, i) for i, char in enumerate(target_chars)])

# Prepare data
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

# Step (b): Build the Seq2Seq Model
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=64, epochs=300)


# Step (d): Create encoder and decoder models for inference
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Step (e): Define functions for translating new sentences
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.0

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]

    return decoded_sentence

# Test the model
for seq_index in range(len(input_texts)):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("Input:", input_texts[seq_index])
    print("Predicted translation:", decoded_sentence)


Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 1.6267
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step - loss: 1.6183
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - loss: 1.6113
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step - loss: 1.6042
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - loss: 1.5963
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - loss: 1.5860
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - loss: 1.5681
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 1.5373
Epoch 9/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - loss: 1.5223
Epoch 10/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - loss: 1.5185
Epoch 11/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 1.5113
Epoch 12/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - loss: 1.4983
Epoch 13/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - loss: 1.4804
Epoch 14/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - loss: 1.4609
Epoch 15/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - loss: 1.4436
Epoch 16/3